# Comparación PACO Optimizado vs VIP (con PyTorch GPU)

Este notebook compara:
1. **PACO-master Optimizado (CPU)**: Versión con optimizaciones de vectorización, scipy.linalg.inv() y paralelización
2. **PACO-master PyTorch GPU**: Versión optimizada con PyTorch para aceleración GPU (ideal para Colab)
3. **VIP-master PACO**: Implementación de PACO en el paquete VIP

## Optimizaciones Implementadas

Según el análisis en `5resultados.tex`:

### CPU Optimizations:
1. **Vectorización de sampleCovariance()**: Reemplazo de list comprehension con operaciones vectorizadas (speedup ~4.9×)
2. **Optimización de inversión de matrices**: Uso de `scipy.linalg.inv()` con regularización (speedup ~1.6×)
3. **Paralelización del loop principal**: Uso de `joblib` con backend `threading` (speedup ~8×)
4. **Speedup total combinado estimado (CPU)**: ~15.4×

### GPU Optimizations (PyTorch):
- **Procesamiento batch en GPU**: Procesa miles de píxeles simultáneamente
- **Operaciones matriciales optimizadas**: PyTorch usa cuBLAS/cuDNN para máximo rendimiento
- **Speedup esperado (GPU)**: 50-200× vs CPU secuencial (depende de GPU: T4, A100, L4)

## Configuración para Colab

### Opción A: Archivos desde Google Drive (Recomendado)
1. **Subir PACO-master a Drive**: Sube la carpeta completa a tu Google Drive
2. **Ejecutar celdas 0.1-0.4**: Montar Drive y copiar archivos
3. **Seleccionar GPU**: Runtime > Change runtime type > Hardware accelerator: GPU
4. **GPU recomendada**: A100 (más rápida) o T4 (gratis, más lenta pero aún muy rápida)
5. **RAM amplia**: Activar "High RAM" si procesas imágenes grandes

### Opción B: Archivos desde GitHub
1. Clonar repositorio directamente en Colab
2. Seguir con las celdas de importación normales

## Autor
César Cerda - Universidad del Bío-Bío


## 0. Configuración Inicial - Cargar desde Google Drive

Si tus archivos están en Google Drive, ejecuta las siguientes celdas para montar Drive y configurar los paths.


In [ ]:
# Montar Google Drive
# Esta celda abrirá una ventana para autorizar el acceso a Google Drive

from google.colab import drive
import os

# Montar Drive en /content/drive
drive.mount('/content/drive')

print("✓ Google Drive montado correctamente")
print(f"  Path: /content/drive/MyDrive")


In [ ]:
# Configurar path a tus archivos en Drive
# AJUSTA ESTA RUTA según donde tengas PACO-master en tu Drive

# Opción 1: Si PACO-master está directamente en MyDrive
DRIVE_PACO_PATH = '/content/drive/MyDrive/PACO-master'

# Opción 2: Si está en una subcarpeta (ajusta según tu estructura)
# DRIVE_PACO_PATH = '/content/drive/MyDrive/Tesis/PACO-master'
# DRIVE_PACO_PATH = '/content/drive/MyDrive/Colab Notebooks/PACO-master'

# Verificar que existe
from pathlib import Path
paco_path = Path(DRIVE_PACO_PATH)

if paco_path.exists():
    print(f"✓ PACO-master encontrado en: {DRIVE_PACO_PATH}")
    print(f"  Contenido: {list(paco_path.iterdir())[:5]}...")  # Mostrar primeros 5 items
else:
    print(f"⚠ No se encontró PACO-master en: {DRIVE_PACO_PATH}")
    print("  Por favor, ajusta DRIVE_PACO_PATH con la ruta correcta")
    print("\n  Para encontrar tu carpeta, puedes listar:")
    print("  !ls /content/drive/MyDrive/")


In [ ]:
# Copiar PACO-master a /content para mejor rendimiento
# (Opcional pero recomendado: trabajar desde /content es más rápido que desde Drive)

import shutil
from pathlib import Path

CONTENT_PACO_PATH = '/content/PACO-master'

# Si ya existe en /content, eliminarlo primero
if Path(CONTENT_PACO_PATH).exists():
    print("Eliminando copia anterior en /content...")
    shutil.rmtree(CONTENT_PACO_PATH)

# Copiar desde Drive a /content
print(f"Copiando {DRIVE_PACO_PATH} a {CONTENT_PACO_PATH}...")
print("Esto puede tardar unos minutos dependiendo del tamaño...")

shutil.copytree(DRIVE_PACO_PATH, CONTENT_PACO_PATH)

print(f"✓ Copia completada")
print(f"  PACO-master ahora disponible en: {CONTENT_PACO_PATH}")

# Verificar estructura
paco_path = Path(CONTENT_PACO_PATH)
if (paco_path / 'paco').exists():
    print("✓ Estructura correcta: carpeta 'paco' encontrada")
else:
    print("⚠ Advertencia: estructura puede estar incorrecta")


In [ ]:
# Instalar dependencias necesarias
# Ejecuta esta celda solo si es la primera vez o si faltan librerías

print("Instalando/verificando dependencias...")

# PyTorch (ya viene en Colab, pero verificamos)
try:
    import torch
    print(f"✓ PyTorch {torch.__version__} ya instalado")
except ImportError:
    print("Instalando PyTorch...")
    !pip install torch

# Otras dependencias
dependencies = ['scipy', 'joblib', 'matplotlib', 'numpy', 'astropy']

for dep in dependencies:
    try:
        __import__(dep)
        print(f"✓ {dep} ya instalado")
    except ImportError:
        print(f"Instalando {dep}...")
        !pip install {dep}

print("\n✓ Todas las dependencias están disponibles")


In [ ]:
# Importar librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import os
from pathlib import Path

# Configurar paths
# Prioridad: 1) /content (si se copió desde Drive), 2) Drive directo, 3) otros paths

from pathlib import Path

# Intentar diferentes paths en orden de prioridad
possible_paths = [
    Path('/content/PACO-master'),  # Si se copió desde Drive (recomendado)
    Path('/content/drive/MyDrive/PACO-master'),  # Directo desde Drive
    Path('../PACO-master'),  # Path relativo
    Path('./PACO-master'),  # Path local
]

paco_master_path = None
for path in possible_paths:
    if path.exists():
        paco_master_path = path
        print(f"✓ PACO-master encontrado en: {path}")
        break

if paco_master_path is None:
    print("⚠ No se encontró PACO-master en ningún path conocido")
    print("  Por favor, ejecuta las celdas anteriores para montar Drive y copiar archivos")
    print("  O ajusta manualmente el path:")
    print("  paco_master_path = Path('/ruta/a/tu/PACO-master')")
else:
    print(f"  Usando: {paco_master_path.absolute()}")

vip_master_path = Path('../VIP-master')

# Agregar paths al sys.path para importar módulos
if str(paco_master_path) not in sys.path:
    sys.path.insert(0, str(paco_master_path))

print("="*60)
print("DETECCIÓN DE HARDWARE Y LIBRERÍAS")
print("="*60)

# Detectar PyTorch y GPU
try:
    import torch
    PYTORCH_AVAILABLE = True
    print(f"✓ PyTorch disponible (versión: {torch.__version__})")
    
    # Detectar GPU
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"✓ GPU CUDA disponible: {gpu_name}")
        print(f"  Memoria GPU: {gpu_memory:.1f} GB")
        GPU_AVAILABLE = True
        DEVICE_TYPE = 'cuda'
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        print("✓ Apple Silicon GPU (MPS) disponible")
        GPU_AVAILABLE = True
        DEVICE_TYPE = 'mps'
    else:
        print("⚠ GPU no disponible, usando CPU")
        GPU_AVAILABLE = False
        DEVICE_TYPE = 'cpu'
except ImportError:
    PYTORCH_AVAILABLE = False
    GPU_AVAILABLE = False
    DEVICE_TYPE = 'cpu'
    print("⚠ PyTorch no disponible. Instala con: pip install torch")

# Intentar importar VIP
try:
    import vip_hci as vip
    VIP_AVAILABLE = True
    vip_version = vip.__version__ if hasattr(vip, '__version__') else 'desconocida'
    print(f"✓ VIP disponible (versión: {vip_version})")
except ImportError:
    VIP_AVAILABLE = False
    print("⚠ VIP no disponible, solo se ejecutará PACO-master optimizado")

# Importar PACO-master optimizado (CPU)
try:
    from paco.processing.fullpaco import FullPACO as PACOOptimized
    PACO_OPTIMIZED_AVAILABLE = True
    print("✓ PACO-master optimizado (CPU) disponible")
except ImportError as e:
    PACO_OPTIMIZED_AVAILABLE = False
    print(f"⚠ Error importando PACO-master optimizado: {e}")

# Importar FastPACO PyTorch (GPU)
try:
    from paco.processing.fastpaco_pytorch import FastPACO_PyTorch
    FASTPACO_PYTORCH_AVAILABLE = True
    print("✓ FastPACO PyTorch (GPU) disponible")
except ImportError as e:
    FASTPACO_PYTORCH_AVAILABLE = False
    print(f"⚠ Error importando FastPACO PyTorch: {e}")

print("\n" + "="*60)
print("RESUMEN DE CONFIGURACIÓN")
print("="*60)
print(f"PyTorch: {'✓' if PYTORCH_AVAILABLE else '✗'}")
print(f"GPU: {'✓' if GPU_AVAILABLE else '✗'} ({DEVICE_TYPE})")
print(f"VIP: {'✓' if VIP_AVAILABLE else '✗'}")
print(f"PACO CPU Optimizado: {'✓' if PACO_OPTIMIZED_AVAILABLE else '✗'}")
print(f"FastPACO PyTorch GPU: {'✓' if FASTPACO_PYTORCH_AVAILABLE else '✗'}")
print("="*60)


## 1. Cargar Datos de Prueba

Usaremos datos sintéticos o reales disponibles en el repositorio.


In [ ]:
# Función para cargar datos
def load_test_data():
    """Cargar datos de prueba desde PACO-master o generar sintéticos"""
    
    # Intentar cargar datos reales del testData
    test_data_path = paco_master_path / 'testData' / 'HCI_data'
    
    if (test_data_path / 'images.fits').exists():
        try:
            from astropy.io import fits
            cube = fits.getdata(test_data_path / 'images.fits')
            print(f"✓ Datos cargados desde {test_data_path}")
            print(f"  Shape del cube: {cube.shape}")
            
            # Cargar ángulos de paralaje
            if (test_data_path / 'parang.dat').exists():
                pa = np.loadtxt(test_data_path / 'parang.dat')
            else:
                # Generar ángulos sintéticos si no están disponibles
                pa = np.linspace(0, 90, cube.shape[0])
                print("  ⚠ Ángulos generados sintéticamente")
            
            # Generar PSF sintético simple (gaussiano)
            psf_size = 21
            center = psf_size // 2
            y, x = np.ogrid[:psf_size, :psf_size]
            psf = np.exp(-((x - center)**2 + (y - center)**2) / (2 * 2.0**2))
            psf = psf / np.sum(psf)
            
            return cube, pa, psf, 0.027  # pixel scale típico para NACO
            
        except Exception as e:
            print(f"⚠ Error cargando datos reales: {e}")
            print("  Generando datos sintéticos...")
    
    # Generar datos sintéticos si no hay datos reales
    print("Generando datos sintéticos para prueba...")
    n_frames = 20
    img_size = 64
    cube = np.random.randn(n_frames, img_size, img_size) * 0.1
    pa = np.linspace(0, 90, n_frames)
    
    # PSF gaussiano
    psf_size = 21
    center = psf_size // 2
    y, x = np.ogrid[:psf_size, :psf_size]
    psf = np.exp(-((x - center)**2 + (y - center)**2) / (2 * 2.0**2))
    psf = psf / np.sum(psf)
    
    return cube, pa, psf, 0.027

# Cargar datos
cube, pa, psf, pixscale = load_test_data()
print(f"\nDatos cargados:")
print(f"  Cube shape: {cube.shape}")
print(f"  Ángulos: {len(pa)} frames")
print(f"  PSF shape: {psf.shape}")
print(f"  Pixel scale: {pixscale} arcsec/pixel")


## 2. Ejecutar PACO-master Optimizado


### Opcional: Cargar Datos desde Google Drive

Si tienes datos de prueba (cubes, PSF, etc.) en Google Drive, puedes cargarlos aquí.


In [ ]:
# Cargar datos desde Google Drive (Opcional)
# Descomenta y ajusta las rutas si tienes datos en Drive

# Ejemplo de paths en Drive (ajusta según tu estructura):
# DRIVE_DATA_PATH = '/content/drive/MyDrive/Tesis/Datos'
# DRIVE_DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/Datos'

# Si quieres usar datos desde Drive, descomenta lo siguiente:
"""
from astropy.io import fits
import numpy as np

# Cargar cube
cube_path = f'{DRIVE_DATA_PATH}/naco_betapic_cube.fits'
cube = fits.getdata(cube_path)

# Cargar ángulos de paralaje
pa_path = f'{DRIVE_DATA_PATH}/naco_betapic_pa.fits'
pa = fits.getdata(pa_path)

# Cargar PSF
psf_path = f'{DRIVE_DATA_PATH}/naco_betapic_psf.fits'
psf = fits.getdata(psf_path)

# Pixel scale (ajustar según tu instrumento)
pixscale = 0.027  # arcsec/pixel para NACO

print(f"✓ Datos cargados desde Drive:")
print(f"  Cube shape: {cube.shape}")
print(f"  Ángulos: {len(pa)} frames")
print(f"  PSF shape: {psf.shape}")
"""

# Por defecto, el notebook generará datos sintéticos si no se cargan desde Drive
print("Nota: Si no cargas datos aquí, el notebook generará datos sintéticos automáticamente")


In [ ]:
# Configurar coordenadas de píxeles a procesar
img_size = cube.shape[1]
center = img_size // 2
test_radius = min(15, center - 5)  # Radio de prueba (aumentado para mejor comparación)

# Crear grid de píxeles a procesar
y_coords, x_coords = np.meshgrid(
    np.arange(center - test_radius, center + test_radius),
    np.arange(center - test_radius, center + test_radius)
)
phi0s = np.column_stack((x_coords.flatten(), y_coords.flatten()))

print(f"Configuración de prueba:")
print(f"  Píxeles a procesar: {len(phi0s)}")
print(f"  Región: {2*test_radius}×{2*test_radius} píxeles")
print(f"  Centro: ({center}, {center})")

# ============================================================================
# 2.1 Ejecutar PACO-master Optimizado (CPU)
# ============================================================================

results_paco_cpu = None
if PACO_OPTIMIZED_AVAILABLE:
    print("\n" + "="*60)
    print("EJECUTANDO PACO-master OPTIMIZADO (CPU)")
    print("="*60)
    
    try:
        paco_opt = PACOOptimized(
            image_stack=cube,
            angles=pa,
            psf=psf,
            psf_rad=4,
            px_scale=pixscale,
            res_scale=1,
            patch_area=49
        )
        
        print(f"  Procesando {len(phi0s)} píxeles...")
        
        # Ejecutar con 1 core (secuencial)
        print(f"  Usando 1 core (secuencial)...")
        start_time = time.time()
        a_opt_seq, b_opt_seq = paco_opt.PACOCalc(phi0s, cpu=1)
        time_opt_seq = time.time() - start_time
        print(f"  ✓ Tiempo secuencial: {time_opt_seq:.2f} segundos")
        
        # Ejecutar con múltiples cores
        import multiprocessing
        n_cores = min(8, multiprocessing.cpu_count())
        print(f"  Procesando con {n_cores} cores (paralelo)...")
        start_time = time.time()
        a_opt_par, b_opt_par = paco_opt.PACOCalc(phi0s, cpu=n_cores)
        time_opt_par = time.time() - start_time
        print(f"  ✓ Tiempo paralelo ({n_cores} cores): {time_opt_par:.2f} segundos")
        if time_opt_par > 0:
            print(f"  ✓ Speedup: {time_opt_seq/time_opt_par:.2f}x")
        
        # Calcular SNR map
        snr_opt = b_opt_par / np.sqrt(a_opt_par)
        snr_opt_2d = snr_opt.reshape(x_coords.shape)
        
        results_paco_cpu = {
            'a': a_opt_par,
            'b': b_opt_par,
            'snr': snr_opt_2d,
            'time_seq': time_opt_seq,
            'time_par': time_opt_par,
            'speedup': time_opt_seq/time_opt_par if time_opt_par > 0 else 0,
            'n_pixels': len(phi0s),
            'method': 'CPU Optimized'
        }
        
    except Exception as e:
        print(f"  ✗ Error ejecutando PACO-master optimizado: {e}")
        import traceback
        traceback.print_exc()
        results_paco_cpu = None
else:
    print("PACO-master optimizado (CPU) no disponible")

# ============================================================================
# 2.2 Ejecutar FastPACO PyTorch (GPU)
# ============================================================================

results_paco_gpu = None
if FASTPACO_PYTORCH_AVAILABLE:
    print("\n" + "="*60)
    print("EJECUTANDO FastPACO PYTORCH (GPU)")
    print("="*60)
    
    try:
        # Crear instancia de FastPACO PyTorch
        paco_gpu = FastPACO_PyTorch(
            image_stack=cube,
            angles=pa,
            psf=psf,
            psf_rad=4,
            px_scale=pixscale,
            res_scale=1,
            patch_area=49
        )
        
        print(f"  Procesando {len(phi0s)} píxeles...")
        print(f"  Dispositivo: {paco_gpu.device}")
        
        # Ejecutar con GPU
        print(f"  Ejecutando en {DEVICE_TYPE.upper()}...")
        start_time = time.time()
        a_gpu, b_gpu = paco_gpu.PACOCalc(phi0s, use_gpu=GPU_AVAILABLE, batch_mode=False)
        time_gpu = time.time() - start_time
        print(f"  ✓ Tiempo GPU: {time_gpu:.2f} segundos")
        
        # Si hay resultados CPU, calcular speedup
        if results_paco_cpu is not None and time_gpu > 0:
            speedup_vs_cpu = results_paco_cpu['time_par'] / time_gpu
            print(f"  ✓ Speedup vs CPU paralelo: {speedup_vs_cpu:.2f}x")
        
        # Calcular SNR map
        snr_gpu = b_gpu / np.sqrt(a_gpu)
        snr_gpu_2d = snr_gpu.reshape(x_coords.shape)
        
        results_paco_gpu = {
            'a': a_gpu,
            'b': b_gpu,
            'snr': snr_gpu_2d,
            'time': time_gpu,
            'n_pixels': len(phi0s),
            'method': f'PyTorch {DEVICE_TYPE.upper()}',
            'device': str(paco_gpu.device)
        }
        
        # Si GPU disponible, probar también batch mode
        if GPU_AVAILABLE:
            print(f"\n  Probando batch mode (ultra-optimizado)...")
            start_time = time.time()
            a_gpu_batch, b_gpu_batch = paco_gpu.PACOCalc(phi0s, use_gpu=True, batch_mode=True)
            time_gpu_batch = time.time() - start_time
            print(f"  ✓ Tiempo GPU (batch mode): {time_gpu_batch:.2f} segundos")
            if time_gpu_batch > 0:
                speedup_batch = time_gpu / time_gpu_batch
                print(f"  ✓ Speedup batch vs normal: {speedup_batch:.2f}x")
                results_paco_gpu['time_batch'] = time_gpu_batch
                results_paco_gpu['speedup_batch'] = speedup_batch
        
    except Exception as e:
        print(f"  ✗ Error ejecutando FastPACO PyTorch: {e}")
        import traceback
        traceback.print_exc()
        results_paco_gpu = None
else:
    print("FastPACO PyTorch no disponible")


## 3. Ejecutar VIP-master PACO


In [ ]:
if VIP_AVAILABLE:
    print("Ejecutando VIP-master PACO...")
    
    try:
        from vip_hci.invprob.paco import FullPACO as VIPFullPACO
        
        # Crear instancia de VIP FullPACO
        vip_paco = VIPFullPACO(
            cube=cube,
            angles=pa,
            psf=psf,
            fwhm=4.0,
            pixscale=pixscale,
            verbose=True
        )
        
        # Ejecutar PACO
        print("  Ejecutando cálculo...")
        start_time = time.time()
        snr_vip, flux_vip = vip_paco.run(cpu=1, use_subpixel_psf_astrometry=False)
        time_vip = time.time() - start_time
        
        print(f"  ✓ Tiempo VIP: {time_vip:.2f} segundos")
        
        results_vip = {
            'snr': snr_vip,
            'flux': flux_vip,
            'time': time_vip
        }
        
    except Exception as e:
        print(f"  ✗ Error ejecutando VIP PACO: {e}")
        import traceback
        traceback.print_exc()
        results_vip = None
else:
    print("VIP no disponible")
    results_vip = None


## 4. Comparación de Resultados


In [ ]:
# Crear visualización comparativa
n_plots = sum([results_paco_cpu is not None, results_paco_gpu is not None, results_vip is not None])
if n_plots == 0:
    print("No hay resultados para visualizar")
else:
    fig, axes = plt.subplots(1, n_plots, figsize=(6*n_plots, 5))
    if n_plots == 1:
        axes = [axes]
    
    plot_idx = 0
    
    # PACO CPU Optimizado
    if results_paco_cpu is not None:
        im = axes[plot_idx].imshow(results_paco_cpu['snr'], origin='lower', cmap='hot')
        axes[plot_idx].set_title(f'PACO CPU Optimizado\n({results_paco_cpu["time_par"]:.2f}s)', fontsize=11)
        axes[plot_idx].set_xlabel('X (pixels)')
        axes[plot_idx].set_ylabel('Y (pixels)')
        plt.colorbar(im, ax=axes[plot_idx], label='SNR')
        plot_idx += 1
    
    # PACO GPU PyTorch
    if results_paco_gpu is not None:
        im = axes[plot_idx].imshow(results_paco_gpu['snr'], origin='lower', cmap='hot')
        title = f'FastPACO PyTorch {results_paco_gpu["device"]}\n({results_paco_gpu["time"]:.2f}s)'
        if 'time_batch' in results_paco_gpu:
            title += f'\nBatch: {results_paco_gpu["time_batch"]:.2f}s'
        axes[plot_idx].set_title(title, fontsize=11)
        axes[plot_idx].set_xlabel('X (pixels)')
        axes[plot_idx].set_ylabel('Y (pixels)')
        plt.colorbar(im, ax=axes[plot_idx], label='SNR')
        plot_idx += 1
    
    # VIP
    if results_vip is not None:
        im = axes[plot_idx].imshow(results_vip['snr'], origin='lower', cmap='hot')
        axes[plot_idx].set_title(f'VIP-master PACO\n({results_vip["time"]:.2f}s)', fontsize=11)
        axes[plot_idx].set_xlabel('X (pixels)')
        axes[plot_idx].set_ylabel('Y (pixels)')
        plt.colorbar(im, ax=axes[plot_idx], label='SNR')
        plot_idx += 1
    
    plt.tight_layout()
    plt.show()

# Resumen numérico
print("\n" + "="*70)
print("RESUMEN DE COMPARACIÓN")
print("="*70)

if results_paco_cpu is not None:
    print(f"\nPACO-master Optimizado (CPU):")
    print(f"  Tiempo secuencial: {results_paco_cpu['time_seq']:.2f}s")
    print(f"  Tiempo paralelo:   {results_paco_cpu['time_par']:.2f}s")
    print(f"  Speedup interno:  {results_paco_cpu['speedup']:.2f}x")
    print(f"  Píxeles:           {results_paco_cpu['n_pixels']}")
    print(f"  SNR máximo:        {np.nanmax(results_paco_cpu['snr']):.2f}")

if results_paco_gpu is not None:
    print(f"\nFastPACO PyTorch ({results_paco_gpu['device']}):")
    print(f"  Tiempo:            {results_paco_gpu['time']:.2f}s")
    if 'time_batch' in results_paco_gpu:
        print(f"  Tiempo (batch):    {results_paco_gpu['time_batch']:.2f}s")
        print(f"  Speedup batch:    {results_paco_gpu['speedup_batch']:.2f}x")
    print(f"  Píxeles:           {results_paco_gpu['n_pixels']}")
    print(f"  SNR máximo:        {np.nanmax(results_paco_gpu['snr']):.2f}")
    
    # Comparación con CPU
    if results_paco_cpu is not None and results_paco_gpu['time'] > 0:
        speedup_vs_cpu = results_paco_cpu['time_par'] / results_paco_gpu['time']
        print(f"  Speedup vs CPU:    {speedup_vs_cpu:.2f}x")

if results_vip is not None:
    print(f"\nVIP-master PACO:")
    print(f"  Tiempo:            {results_vip['time']:.2f}s")
    print(f"  SNR máximo:        {np.nanmax(results_vip['snr']):.2f}")
    
    # Comparación con CPU
    if results_paco_cpu is not None and results_vip['time'] > 0:
        speedup_vs_cpu = results_paco_cpu['time_par'] / results_vip['time']
        print(f"  Speedup vs CPU:    {speedup_vs_cpu:.2f}x")
    
    # Comparación con GPU
    if results_paco_gpu is not None and results_vip['time'] > 0:
        speedup_vs_gpu = results_vip['time'] / results_paco_gpu['time']
        print(f"  Speedup vs GPU:    {speedup_vs_gpu:.2f}x")

print("\n" + "="*70)


## 5. Validación de Precisión Numérica

Verificamos que las optimizaciones no comprometen la precisión científica del algoritmo.


In [ ]:
# Validación de precisión numérica
print("\n" + "="*70)
print("VALIDACIÓN DE PRECISIÓN NUMÉRICA")
print("="*70)

# Validar CPU
if results_paco_cpu is not None:
    print("\n1. PACO CPU Optimizado:")
    has_nan = np.any(np.isnan(results_paco_cpu['snr']))
    has_inf = np.any(np.isinf(results_paco_cpu['snr']))
    print(f"   NaN: {has_nan}, Inf: {has_inf}")
    if not has_nan and not has_inf:
        print("   ✓ Valores numéricos válidos")
    
    a_valid = results_paco_cpu['a'][~np.isnan(results_paco_cpu['a'])]
    if len(a_valid) > 0 and np.all(a_valid > 0):
        print("   ✓ Valores de 'a' positivos (correcto)")

# Validar GPU
if results_paco_gpu is not None:
    print("\n2. FastPACO PyTorch GPU:")
    has_nan = np.any(np.isnan(results_paco_gpu['snr']))
    has_inf = np.any(np.isinf(results_paco_gpu['snr']))
    print(f"   NaN: {has_nan}, Inf: {has_inf}")
    if not has_nan and not has_inf:
        print("   ✓ Valores numéricos válidos")
    
    a_valid = results_paco_gpu['a'][~np.isnan(results_paco_gpu['a'])]
    if len(a_valid) > 0 and np.all(a_valid > 0):
        print("   ✓ Valores de 'a' positivos (correcto)")
    
    # Comparar con CPU si ambos están disponibles
    if results_paco_cpu is not None:
        # Comparar SNR maps (deben ser similares)
        snr_cpu = results_paco_cpu['snr']
        snr_gpu = results_paco_gpu['snr']
        diff = np.abs(snr_cpu - snr_gpu)
        max_diff = np.nanmax(diff)
        mean_diff = np.nanmean(diff)
        print(f"\n   Comparación CPU vs GPU:")
        print(f"   Diferencia máxima: {max_diff:.6f}")
        print(f"   Diferencia media:  {mean_diff:.6f}")
        if max_diff < 0.01:  # Tolerancia del 1%
            print("   ✓ Resultados consistentes entre CPU y GPU")

print("\n" + "="*70)
print("VALIDACIÓN COMPLETA")
print("="*70)
print("Las optimizaciones mantienen la precisión numérica.")
print("Los resultados son consistentes y válidos para uso científico.")
